In [ ]:
# default_exp utils

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
# export 
import math
import cv2
import numpy as np
import os
import gdown

def get_grasp_line(theta, center, depth):
    """Generate grasping two endpoints of grasping line(8cm).

    Inputs:
      theta: degree
      center: x, y coordinate
      depth: depth image (mm)
    """
    depth = depth/1000.0
    if depth[center[0], center[1]] < 0.1:
        dis = 0.356 - 0.199
    else:
        dis = depth[center[0], center[1]] - 0.199

    length = int((148 - int(dis*50.955))/2)

    rad = math.radians(theta)
    if theta < 90:
        x1 = int(center[1] + length*abs(math.cos(rad)))
        y1 = int(center[0] - length*abs(math.sin(rad)))
        x2 = int(center[1] - length*abs(math.cos(rad)))
        y2 = int(center[0] + length*abs(math.sin(rad)))
    else:
        x1 = int(length*abs(math.cos(rad)) + center[1])
        y1 = int(length*abs(math.sin(rad)) + center[0])
        x2 = int(center[1] - length*abs(math.cos(rad)))
        y2 = int(center[0] - length*abs(math.sin(rad)))

    p1 = (x1, y1)
    p2 = (x2, y2)

    return p1, p2

def get_affordancemap(predict, depth):
    """Generate grasping point and affordancemap.

    Inputs:
      predict: output of HANet
      depth: depth image (mm)
    """
    Max = []
    Re = []
    Angle = [90,135,0,45]
    height = depth.shape[0]
    width = depth.shape[1]
    re = np.zeros((4, height, width))

    for i in range(4):
        x, y = np.where(predict[0][i] == np.max(predict[0][i]))
        re[i] = cv2.resize(predict[0][i], (width, height))
        Max.append(np.max(predict[0][i]))
        Re.append(re[i])

    theta = Angle[Max.index(max(Max))]
    graspable = re[Max.index(max(Max))]
    graspable = cv2.resize(graspable, (640, 480))
    depth = cv2.resize(depth, (640, 480))
    graspable [depth==0] = 0
    graspable[graspable>=1] = 0.99999
    graspable[graspable<0] = 0
    graspable = cv2.GaussianBlur(graspable, (7, 7), 0)
    affordanceMap = (graspable/np.max(graspable)*255).astype(np.uint8)
    affordanceMap = cv2.applyColorMap(affordanceMap, cv2.COLORMAP_JET)
    affordanceMap = affordanceMap[:,:,[2,1,0]]

    gray = cv2.cvtColor(affordanceMap, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (11, 11), 0)
    binaryIMG = cv2.Canny(blurred, 20, 160)
    contours, _ = cv2.findContours(binaryIMG, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    i = 0
    point_x = 0
    point_y = 0
    cX = 0
    cY = 0
    x = 0
    y = 0

    for c in contours:
        M = cv2.moments(c)
        if(M["m00"]!=0):
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            zc = depth[cY, cX]/1000
            i += 1
            point_x += cX
            point_y += cY

    if i != 0:
        x = int(point_x / i)
        y = int(point_y / i)

    return affordanceMap, x, y, theta

def get_model():
    dataset_url = 'https://drive.google.com/u/1/uc?id=1jnlGIrqIXAIzOUjv2poJZMxxfzmtg1ET'
    dataset_name = 'HANet'
    path = os.path.abspath(os.getcwd())
    if not os.path.isfile(path+'/'+dataset_name + '.pth'):
        gdown.download(dataset_url, output=dataset_name + '.pth', quiet=False)

    return path + '/HANet.pth'